# BentoML PyTorch MNIST Tutorial

Link to source code: https://github.com/bentoml/gallery/tree/main/pytorch_mnist_demo/

Install required dependencies:

In [1]:
!pip install -r requirements.txt

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import transforms
from sklearn.model_selection import KFold

import bentoml

# reproducible setup for testing
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

def _dataloader_init_fn(worker_id):
    np.random.seed(seed)


## Training and Saving the model

First we define a simple PyTorch network and some helper functions

In [3]:
K_FOLDS = 5
NUM_EPOCHS = 5
LOSS_FUNCTION = nn.CrossEntropyLoss()

def get_dataset():
    # Prepare MNIST dataset by concatenating Train/Test part; we split later.
    train_set = MNIST(os.getcwd(), download=True, transform=transforms.ToTensor(), train=True)
    test_set = MNIST(os.getcwd(), download=True, transform=transforms.ToTensor(), train=False)
    return train_set, test_set


class SimpleConvNet(nn.Module):
    '''
    Simple Convolutional Neural Network
    '''
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(26 * 26 * 10, 50),
            nn.ReLU(),
            nn.Linear(50, 20),
            nn.ReLU(),
            nn.Linear(20, 10)
        )

    def forward(self, x):
        return self.layers(x)

    def predict(self, inp):
        """predict digit for input"""
        self.eval()
        with torch.no_grad():
            raw_output = self(inp)
            _, pred = torch.max(raw_output, 1)
            return pred

def train_epoch(model, optimizer, loss_function, train_loader, epoch, device="cpu"):
    # Mark training flag
    model.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        if batch_idx % 499 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(inputs), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test_model(model, test_loader, device="cpu"):
    correct, total = 0, 0
    model.eval()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    return correct, total


In [4]:
# load data
train_set, test_set = get_dataset()
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=10,
    sampler=torch.utils.data.RandomSampler(test_set),
    worker_init_fn=_dataloader_init_fn,
)


/home/larme/notebooks/venv/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### Cross Validation

We can do some cross validation and the results can be saved with the model as metadata


In [5]:
def cross_validate(dataset, epochs=NUM_EPOCHS, k_folds=K_FOLDS):
    results = {}

    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=k_folds, shuffle=True)

    print('--------------------------------')

    # K-fold Cross Validation model evaluation
    for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

        print(f'FOLD {fold}')
        print('--------------------------------')
    
        # Sample elements randomly from a given list of ids, no replacement.
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

        # Define data loaders for training and testing data in this fold
        train_loader = torch.utils.data.DataLoader(
            dataset, 
            batch_size=10,
            sampler=train_subsampler,
            worker_init_fn=_dataloader_init_fn,
        )
        test_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=10,
            sampler=test_subsampler,
            worker_init_fn=_dataloader_init_fn,
        )

        # Train this fold
        model = SimpleConvNet()
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
        loss_function = nn.CrossEntropyLoss()
        for epoch in range(epochs):
            train_epoch(model, optimizer, loss_function, train_loader, epoch)

        # Evaluation for this fold
        correct, total = test_model(model, test_loader)
        print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
        print('--------------------------------')
        results[fold] = 100.0 * (correct / total)

    # Print fold results
    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {K_FOLDS} FOLDS')
    print('--------------------------------')
    sum = 0.0
    for key, value in results.items():
        print(f'Fold {key}: {value} %')
        sum += value

    print(f'Average: {sum/len(results.items())} %')

    return results


In [6]:
cv_results = cross_validate(train_set, epochs=1)

--------------------------------
FOLD 0
--------------------------------
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.273126
Train Epoch: 0 [4990/60000 (10%)]	Loss: 1.362214
Train Epoch: 0 [9980/60000 (21%)]	Loss: 0.411741
Train Epoch: 0 [14970/60000 (31%)]	Loss: 1.001374
Train Epoch: 0 [19960/60000 (42%)]	Loss: 0.102851
Train Epoch: 0 [24950/60000 (52%)]	Loss: 0.511350
Train Epoch: 0 [29940/60000 (62%)]	Loss: 0.054312
Train Epoch: 0 [34930/60000 (73%)]	Loss: 0.229270
Train Epoch: 0 [39920/60000 (83%)]	Loss: 0.369209
Train Epoch: 0 [44910/60000 (94%)]	Loss: 0.130033
Accuracy for fold 0: 91 %
--------------------------------
FOLD 1
--------------------------------
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.327491
Train Epoch: 0 [4990/60000 (10%)]	Loss: 1.318107
Train Epoch: 0 [9980/60000 (21%)]	Loss: 0.485992
Train Epoch: 0 [14970/60000 (31%)]	Loss: 1.216311
Train Epoch: 0 [19960/60000 (42%)]	Loss: 0.419983
Train Epoch: 0 [24950/60000 (52%)]	Loss: 0.758953
Train Epoch: 0 [29940/60000 (62%)]	Loss: 0

### training the model

In [7]:
def train(dataset, epochs=NUM_EPOCHS, device="cpu"):

    train_sampler = torch.utils.data.RandomSampler(dataset)
    train_loader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=10,
        sampler=train_sampler,
        worker_init_fn=_dataloader_init_fn,
    )
    model = SimpleConvNet()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    loss_function = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        train_epoch(model, optimizer, loss_function, train_loader, epoch, device)
    return model

In [8]:
trained_model = train(train_set)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.371420
Train Epoch: 0 [4990/60000 (8%)]	Loss: 1.000866
Train Epoch: 0 [9980/60000 (17%)]	Loss: 1.110684
Train Epoch: 0 [14970/60000 (25%)]	Loss: 0.737674
Train Epoch: 0 [19960/60000 (33%)]	Loss: 0.364441
Train Epoch: 0 [24950/60000 (42%)]	Loss: 0.390488
Train Epoch: 0 [29940/60000 (50%)]	Loss: 0.282015
Train Epoch: 0 [34930/60000 (58%)]	Loss: 0.199643
Train Epoch: 0 [39920/60000 (67%)]	Loss: 0.367593
Train Epoch: 0 [44910/60000 (75%)]	Loss: 0.554671
Train Epoch: 0 [49900/60000 (83%)]	Loss: 0.017668
Train Epoch: 0 [54890/60000 (91%)]	Loss: 0.086637
Train Epoch: 0 [59880/60000 (100%)]	Loss: 0.249381
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.222885
Train Epoch: 1 [4990/60000 (8%)]	Loss: 0.288009
Train Epoch: 1 [9980/60000 (17%)]	Loss: 0.151059
Train Epoch: 1 [14970/60000 (25%)]	Loss: 0.118367
Train Epoch: 1 [19960/60000 (33%)]	Loss: 0.757163
Train Epoch: 1 [24950/60000 (42%)]	Loss: 0.100752
Train Epoch: 1 [29940/60000 (50%)]	Loss: 0.131567
Train Epoch: 1 

### saving the model with some metadata

In [ ]:
correct, total = test_model(trained_model, test_loader)
metadata = {
    "accuracy": float(correct)/total,
    "cv_stats": cv_results,
}

tag = bentoml.pytorch.save(
    "pytorch_mnist_demo",
    trained_model,
    metadata=metadata,
)


## Create a BentoML Service for serving the model

Note: using `%%writefile` here because `bentoml.Service` instance must be created in a separate `.py` file

Even though we have only one model, we can create as many api endpoints as we want. Here we create two end points `predict_ndarray` and `predict_image`

In [20]:
%%writefile service.py

import typing as t

import numpy as np
import PIL.Image
from PIL.Image import Image as PILImage

import bentoml
from bentoml.io import Image
from bentoml.io import NumpyNdarray


mnist_runner = bentoml.pytorch.load_runner(
    "pytorch_mnist_demo",
    name="mnist_runner",
    predict_fn_name="predict",
)

svc = bentoml.Service(
    name="pytorch_mnist_demo",
    runners=[
        mnist_runner,
    ],
)


@svc.api(
    input=NumpyNdarray(dtype="float32", enforce_dtype=True),
    output=NumpyNdarray(dtype="int64"),
)
async def predict_ndarray(
    inp: "np.ndarray[t.Any, np.dtype[t.Any]]",
) -> "np.ndarray[t.Any, np.dtype[t.Any]]":
    assert inp.shape == (28, 28)
    # We are using greyscale image and our PyTorch model expect one
    # extra channel dimension
    inp = np.expand_dims(inp, 0)
    output_tensor = await mnist_runner.async_run(inp)
    return output_tensor.numpy()


@svc.api(input=Image(), output=NumpyNdarray(dtype="int64"))
async def predict_image(f: PILImage) -> "np.ndarray[t.Any, np.dtype[t.Any]]":
    assert isinstance(f, PILImage)
    arr = np.array(f)/255.0
    assert arr.shape == (28, 28)

    # We are using greyscale image and our PyTorch model expect one
    # extra channel dimension
    arr = np.expand_dims(arr, 0).astype("float32")
    output_tensor = await mnist_runner.async_run(arr)
    return output_tensor.numpy()


Writing pytorch_mnist_demo.py


Start a dev model server to test out the service defined above

In [16]:
!bentoml serve pytorch_mnist_demo.py:svc

[08:55:07 PM] INFO     Starting development BentoServer from                    
                       "pytorch_mnist_demo.py:svc"                              
[08:55:07 PM] INFO     Service imported from source:                            
                       bentoml.Service(name="pytorch_mnist_demo",               
                       import_str="pytorch_mnist_demo:svc",                     
                       working_dir="/home/larme/notebooks/gallery/pytorch")     
[08:55:07 PM] INFO     Started server process [21916]               ]8;id=863904;file:///home/larme/notebooks/venv/lib/python3.7/site-packages/uvicorn/server.py\server.py]8;;\:]8;id=301963;file:///home/larme/notebooks/venv/lib/python3.7/site-packages/uvicorn/server.py#84\84]8;;\
              INFO     Waiting for application startup.                 ]8;id=147010;file:///home/larme/notebooks/venv/lib/python3.7/site-packages/uvicorn/lifespan/on.py\on.py]8;;\:]8;id=904060;file:///home/larme/notebook

Now you can use something like:

`curl -H "Content-Type: multipart/form-data" -F'fileobj=@samples/1.png;type=image/png' http://127.0.0.1:5000/predict_image`
    
to send an image to the digit recognition service

## Build a Bento for distribution and deployment

In [ ]:
bentoml.build(
    "service.py:svc",
    include=["*.py"],
    exclude=["tests/"],
    description="file:./README.md",
    python=dict(
        packages=["scikit-learn", "torch", "Pillow"],
    )
)